<a href="https://colab.research.google.com/github/krithikaark0903/PySQLite-LangChain/blob/main/langchain_demo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install necessary requirements
!pip install --upgrade langchain
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
#CSV to SQL conversion
import pandas as pd
import sqlite3

# Read the CSV file containing employee data
df = pd.read_csv("employees.csv")

# Rename columns to match the database schema
df.rename(columns={
    'EMPLOYEE_ID': 'employee_id',
    'FIRST_NAME': 'first_name',
    'LAST_NAME': 'last_name',
    'EMAIL': 'email',
    'PHONE_NUMBER': 'phone_number',
    'HIRE_DATE': 'hire_date',
    'JOB_ID': 'job_id',
    'SALARY': 'salary',
    'COMMISSION_PCT': 'commission_pct',
    'MANAGER_ID': 'manager_id',
    'DEPARTMENT_ID': 'department_id'
}, inplace=True)

# Create a connection to the SQLite database file
conn = sqlite3.connect('employee_db.sqlite')
c = conn.cursor()

# Convert the DataFrame to a SQLite table
df.to_sql('employees', conn, if_exists='replace', index=False)

# Save (commit) the changes and close the connection
conn.commit()
conn.close()


# Create a connection to the SQLite database file
conn = sqlite3.connect('employee_db.sqlite')
c = conn.cursor()

# SQL statement to create a table for employees
create_table_query = '''
    CREATE TABLE IF NOT EXISTS employees (
        employee_id INTEGER PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        email TEXT,
        phone_number TEXT,
        hire_date TEXT,
        job_id TEXT,
        salary REAL,
        commission_pct REAL,
        manager_id INTEGER,
        department_id INTEGER
    )
'''

# Execute the SQL statement to create the employees table
c.execute(create_table_query)

# Save (commit) the changes and close the connection
conn.commit()
conn.close()


In [ ]:
#checking if db is correctly loaded
import sqlite3

def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)
    conn.close()




In [ ]:
# Example usage for employees table
read_sql_query('SELECT * FROM employees LIMIT 10;', "employee_db.sqlite")

(198, 'Donald', 'OConnell', 'DOCONNEL', '650.507.9833', '21-JUN-07', 'SH_CLERK', 2600, ' - ', '124', 50)
(199, 'Douglas', 'Grant', 'DGRANT', '650.507.9844', '13-JAN-08', 'SH_CLERK', 2600, ' - ', '124', 50)
(200, 'Jennifer', 'Whalen', 'JWHALEN', '515.123.4444', '17-SEP-03', 'AD_ASST', 4400, ' - ', '101', 10)
(201, 'Michael', 'Hartstein', 'MHARTSTE', '515.123.5555', '17-FEB-04', 'MK_MAN', 13000, ' - ', '100', 20)
(202, 'Pat', 'Fay', 'PFAY', '603.123.6666', '17-AUG-05', 'MK_REP', 6000, ' - ', '201', 20)
(203, 'Susan', 'Mavris', 'SMAVRIS', '515.123.7777', '07-JUN-02', 'HR_REP', 6500, ' - ', '101', 40)
(204, 'Hermann', 'Baer', 'HBAER', '515.123.8888', '07-JUN-02', 'PR_REP', 10000, ' - ', '101', 70)
(205, 'Shelley', 'Higgins', 'SHIGGINS', '515.123.8080', '07-JUN-02', 'AC_MGR', 12008, ' - ', '101', 110)
(206, 'William', 'Gietz', 'WGIETZ', '515.123.8181', '07-JUN-02', 'AC_ACCOUNT', 8300, ' - ', '205', 110)
(100, 'Steven', 'King', 'SKING', '515.123.4567', '17-JUN-03', 'AD_PRES', 24000, ' - ', '

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.llms.openai import OpenAI
from langchain import SQLDatabase

# Set your OpenAI GPT API key here
OPENAI_API_KEY = "your_api_key"

db = SQLDatabase.from_uri("sqlite:///employee_db.sqlite")

agent_executor = create_sql_agent(
    llm=OpenAI(api_key=OPENAI_API_KEY, temperature=0),
    toolkit=SQLDatabaseToolkit(db=db, llm=OpenAI(api_key=OPENAI_API_KEY, temperature=0)),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)




In [ ]:
#Question 1
agent_executor.run(
    "List the total number of employees."
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: employees
Thought: I should query the schema of the employees table to see what columns I can use.
Action: sql_db_schema
Action Input: employees
Observation: 
CREATE TABLE employees (
	employee_id INTEGER, 
	first_name TEXT, 
	last_name TEXT, 
	email TEXT, 
	phone_number TEXT, 
	hire_date TEXT, 
	job_id TEXT, 
	salary INTEGER, 
	commission_pct TEXT, 
	manager_id TEXT, 
	department_id INTEGER
)

/*
3 rows from employees table:
employee_id	first_name	last_name	email	phone_number	hire_date	job_id	salary	commission_pct	manager_id	department_id
198	Donald	OConnell	DOCONNEL	650.507.9833	21-JUN-07	SH_CLERK	2600	 - 	124	50
199	Douglas	Grant	DGRANT	650.507.9844	13-JAN-08	SH_CLERK	2600	 - 	124	50
200	Jennifer	Whalen	JWHALEN	515.123.4444	17-SEP-03	AD_ASST	4400	 - 	101	10
*/
Thought: I should query the total number of employees.
Action: sql_db_query
Action Input: SELECT COUNT(*) FROM employees
Observatio

'There are 50 employees.'

In [ ]:
#question 2
agent_executor.run(
    "Retrieve the average salary of employees working in the Sales department."
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: employees
Thought: I should query the schema of the employees table.
Action: sql_db_schema
Action Input: employees
Observation: 
CREATE TABLE employees (
	employee_id INTEGER, 
	first_name TEXT, 
	last_name TEXT, 
	email TEXT, 
	phone_number TEXT, 
	hire_date TEXT, 
	job_id TEXT, 
	salary INTEGER, 
	commission_pct TEXT, 
	manager_id TEXT, 
	department_id INTEGER
)

/*
3 rows from employees table:
employee_id	first_name	last_name	email	phone_number	hire_date	job_id	salary	commission_pct	manager_id	department_id
198	Donald	OConnell	DOCONNEL	650.507.9833	21-JUN-07	SH_CLERK	2600	 - 	124	50
199	Douglas	Grant	DGRANT	650.507.9844	13-JAN-08	SH_CLERK	2600	 - 	124	50
200	Jennifer	Whalen	JWHALEN	515.123.4444	17-SEP-03	AD_ASST	4400	 - 	101	10
*/
Thought: I should query the average salary of employees in the Sales department.
Action: sql_db_query
Action Input: SELECT AVG(salary) FROM employees WHERE depar

'The average salary of employees working in the Sales department is 3721.7391304347825.'

In [ ]:
#Question 3
agent_executor.run(
    "Name the highest paid employee."
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: employees
Thought: I should query the schema of the employees table.
Action: sql_db_schema
Action Input: employees
Observation: 
CREATE TABLE employees (
	employee_id INTEGER, 
	first_name TEXT, 
	last_name TEXT, 
	email TEXT, 
	phone_number TEXT, 
	hire_date TEXT, 
	job_id TEXT, 
	salary INTEGER, 
	commission_pct TEXT, 
	manager_id TEXT, 
	department_id INTEGER
)

/*
3 rows from employees table:
employee_id	first_name	last_name	email	phone_number	hire_date	job_id	salary	commission_pct	manager_id	department_id
198	Donald	OConnell	DOCONNEL	650.507.9833	21-JUN-07	SH_CLERK	2600	 - 	124	50
199	Douglas	Grant	DGRANT	650.507.9844	13-JAN-08	SH_CLERK	2600	 - 	124	50
200	Jennifer	Whalen	JWHALEN	515.123.4444	17-SEP-03	AD_ASST	4400	 - 	101	10
*/
Thought: I should query the employees table for the highest paid employee.
Action: sql_db_query
Action Input: SELECT first_name, last_name, salary FROM employees

'The highest paid employee is Steven King with a salary of 24000.'

In [ ]:
#question 4
agent_executor.run(
    "List 2nd highest paid emplyee and his/her hire date"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: employees
Thought: I should query the schema of the employees table
Action: sql_db_schema
Action Input: employees
Observation: 
CREATE TABLE employees (
	employee_id INTEGER, 
	first_name TEXT, 
	last_name TEXT, 
	email TEXT, 
	phone_number TEXT, 
	hire_date TEXT, 
	job_id TEXT, 
	salary INTEGER, 
	commission_pct TEXT, 
	manager_id TEXT, 
	department_id INTEGER
)

/*
3 rows from employees table:
employee_id	first_name	last_name	email	phone_number	hire_date	job_id	salary	commission_pct	manager_id	department_id
198	Donald	OConnell	DOCONNEL	650.507.9833	21-JUN-07	SH_CLERK	2600	 - 	124	50
199	Douglas	Grant	DGRANT	650.507.9844	13-JAN-08	SH_CLERK	2600	 - 	124	50
200	Jennifer	Whalen	JWHALEN	515.123.4444	17-SEP-03	AD_ASST	4400	 - 	101	10
*/
Thought: I should query the database for the 2nd highest paid employee and their hire date
Action: sql_db_query
Action Input: SELECT first_name, last_name, hire_d

'The 2nd highest paid employee is Neena Kochhar and their hire date is 21-SEP-05.'

In [ ]:
#Question 5
agent_executor.run(
    "what will be the focasted salary if we give 10% hike across board"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: employees
Thought: I should query the schema of the employees table
Action: sql_db_schema
Action Input: employees
Observation: 
CREATE TABLE employees (
	employee_id INTEGER, 
	first_name TEXT, 
	last_name TEXT, 
	email TEXT, 
	phone_number TEXT, 
	hire_date TEXT, 
	job_id TEXT, 
	salary INTEGER, 
	commission_pct TEXT, 
	manager_id TEXT, 
	department_id INTEGER
)

/*
3 rows from employees table:
employee_id	first_name	last_name	email	phone_number	hire_date	job_id	salary	commission_pct	manager_id	department_id
198	Donald	OConnell	DOCONNEL	650.507.9833	21-JUN-07	SH_CLERK	2600	 - 	124	50
199	Douglas	Grant	DGRANT	650.507.9844	13-JAN-08	SH_CLERK	2600	 - 	124	50
200	Jennifer	Whalen	JWHALEN	515.123.4444	17-SEP-03	AD_ASST	4400	 - 	101	10
*/
Thought: I should query the salary of all employees and add 10% to it
Action: sql_db_query
Action Input: SELECT salary, salary * 1.1 AS forecasted_salary FROM emp

'The forecasted salary after a 10% hike across board is 2860.0000000000005 for employees with a salary of 2600, 4840.0 for employees with a salary of 4400, and so on.'